In [2]:
import matplotlib.pyplot as plt # Import the Matplotlib package
import rasterio
import pandas as pd
from datetime import datetime, timedelta
from affine import Affine
from osgeo import osr, gdal
import numpy as np # Import the Numpy package

In [3]:
def getGeoT(extent, nlines, ncols):
    # Compute resolution based on data dimension
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3], 0, -resy]

In [4]:
extent = [-75, -40, -58,  -25 ]
#extent = [-96, -75, -15,  -10 ]

# Define KM_PER_DEGREE
KM_PER_DEGREE = 111.32

resolution = 1

In [5]:
# Read the GRIB file
gribGFS = gdal.Open('../data/GFS/GFS_2021030200+015.grib2')

In [6]:
gribGFS.RasterCount

8

In [7]:
gribGFS.GetProjection()

'GEOGCS["Coordinate System imported from GRIB file",DATUM["unnamed",SPHEROID["Sphere",6371229,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

str

In [8]:
for key in range(1, gribGFS.RasterCount +1):
    var = gribGFS.GetRasterBand(key)
    print(var.GetMetadata())

{'GRIB_COMMENT': 'Pressure [Pa]', 'GRIB_DISCIPLINE': '0(Meteorological)', 'GRIB_ELEMENT': 'PRES', 'GRIB_FORECAST_SECONDS': '54000 sec', 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=0 MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-02T00:00:00Z PROD_STATUS=0(Operational) TYPE=1(Forecast)', 'GRIB_PDS_PDTN': '0', 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '3 0 2 0 96 0 0 1 15 1 0 0 255 0 0', 'GRIB_PDS_TEMPLATE_NUMBERS': '3 0 2 0 96 0 0 0 1 0 0 0 15 1 0 0 0 0 0 255 0 0 0 0 0', 'GRIB_REF_TIME': '  1614643200 sec UTC', 'GRIB_SHORT_NAME': '0-SFC', 'GRIB_UNIT': '[Pa]', 'GRIB_VALID_TIME': '  1614697200 sec UTC'}
{'GRIB_COMMENT': 'Temperature [C]', 'GRIB_DISCIPLINE': '0(Meteorological)', 'GRIB_ELEMENT': 'TMP', 'GRIB_FORECAST_SECONDS': '54000 sec', 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=0 MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-02T00:00:00Z PROD_STATUS=0(Operational) TYPE=1(Forecast)', 'GRIB_PDS_PDTN': '0', 'GRIB_PDS_TEMPLATE_AS

In [152]:
var = gribGFS.GetRasterBand(5)
print(var.GetMetadata())

{'GRIB_COMMENT': 'Geopotential height [gpm]', 'GRIB_DISCIPLINE': '0(Meteorological)', 'GRIB_ELEMENT': 'HGT', 'GRIB_FORECAST_SECONDS': '32400 sec', 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=0 MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-04T06:00:00Z PROD_STATUS=0(Operational) TYPE=1(Forecast)', 'GRIB_PDS_PDTN': '0', 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '3 5 2 0 96 0 0 1 9 100 0 100 255 0 0', 'GRIB_PDS_TEMPLATE_NUMBERS': '3 5 2 0 96 0 0 0 1 0 0 0 9 100 0 0 0 0 100 255 0 0 0 0 0', 'GRIB_REF_TIME': '  1614837600 sec UTC', 'GRIB_SHORT_NAME': '100-ISBL', 'GRIB_UNIT': '[gpm]', 'GRIB_VALID_TIME': '  1614870000 sec UTC'}


In [ ]:
## DAta to reproj

In [13]:
# Read the GRIB file
gribGFS = gdal.Open('../data/GFS/GFS_2021030200+384.grib2')

In [14]:
gribGFS.RasterCount

8

In [143]:
def getPpnBand(grib):
    for band in range(1, grib.RasterCount + 1):
        var = grib.GetRasterBand(band)
        if var.GetMetadata()['GRIB_ELEMENT'] in ("APCP03", "APCP06"):
            print(f"Total precipitation is band {band}")
            return band
    return None

In [144]:
bandNumber = getPpnBand(gribGFS)
if bandNumber == None:
        print("The dataset doesn't contain Total Precipitation variable")

Total precipitation is band 7


In [15]:
for key in range(1, gribGFS.RasterCount +1):
    var = gribGFS.GetRasterBand(key)
    print(var.GetMetadata()['GRIB_ELEMENT'])

PRES
TMP
TMP
RH
UGRD
VGRD
APCP06
APCP384


In [8]:
aa = gribGFS.GetRasterBand(2)
aa.GetMetadata()

{'GRIB_COMMENT': 'Temperature [C]',
 'GRIB_DISCIPLINE': '0(Meteorological)',
 'GRIB_ELEMENT': 'TMP',
 'GRIB_FORECAST_SECONDS': '54000 sec',
 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=0 MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-02T00:00:00Z PROD_STATUS=0(Operational) TYPE=1(Forecast)',
 'GRIB_PDS_PDTN': '0',
 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '0 0 2 0 96 0 0 1 15 1 0 0 255 0 0',
 'GRIB_PDS_TEMPLATE_NUMBERS': '0 0 2 0 96 0 0 0 1 0 0 0 15 1 0 0 0 0 0 255 0 0 0 0 0',
 'GRIB_REF_TIME': '  1614643200 sec UTC',
 'GRIB_SHORT_NAME': '0-SFC',
 'GRIB_UNIT': '[C]',
 'GRIB_VALID_TIME': '  1614697200 sec UTC'}

In [11]:
aa.GetMetadata()['GRIB_REF_TIME'][2:12]

'1614643200'

In [22]:
aa = gribGFS.GetRasterBand(3)
aa.GetMetadata()

{'GRIB_COMMENT': 'Temperature [C]',
 'GRIB_DISCIPLINE': '0(Meteorological)',
 'GRIB_ELEMENT': 'TMP',
 'GRIB_FORECAST_SECONDS': '1382400 sec',
 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=0 MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-02T00:00:00Z PROD_STATUS=0(Operational) TYPE=1(Forecast)',
 'GRIB_PDS_PDTN': '0',
 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '0 0 2 0 96 0 0 1 384 103 0 2 255 0 0',
 'GRIB_PDS_TEMPLATE_NUMBERS': '0 0 2 0 96 0 0 0 1 0 0 1 128 103 0 0 0 0 2 255 0 0 0 0 0',
 'GRIB_REF_TIME': '  1614643200 sec UTC',
 'GRIB_SHORT_NAME': '2-HTGL',
 'GRIB_UNIT': '[C]',
 'GRIB_VALID_TIME': '  1616025600 sec UTC'}

In [9]:
# Read the GRIB file
grib = gdal.Open('../data/GEFS/GEFS_01_2021030200+840.0p50.grib2')

In [10]:
for band in range(1, grib.RasterCount + 1):
    var = grib.GetRasterBand(band)
    if var.GetMetadata()['GRIB_ELEMENT'] == "APCP03":
        print(f"Total precipitation is band {band}")

In [11]:
bandNumber = getPpnBand(grib)
if bandNumber == None:
        print("The dataset doesn't contain Total Precipitation variable")

NameError: name 'getPpnBand' is not defined

In [12]:
for key in range(1, grib.RasterCount +1):
    var = grib.GetRasterBand(key)
    print(f"Band {key} is {var.GetMetadata()['GRIB_ELEMENT']}")

Band 1 is PRES
Band 2 is TMP
Band 3 is RH
Band 4 is UGRD
Band 5 is VGRD
Band 6 is APCP06


In [133]:
aa = grib.GetRasterBand(6)
aa.GetMetadata()

{'GRIB_COMMENT': '06 hr Total precipitation [kg/(m^2)]',
 'GRIB_DISCIPLINE': '0(Meteorological)',
 'GRIB_ELEMENT': 'APCP06',
 'GRIB_FORECAST_SECONDS': '216000 sec',
 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=2(NCEP Ensemble Products) MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-03-02T00:00:00Z PROD_STATUS=0(Operational) TYPE=4(Perturbed_forecast)',
 'GRIB_PDS_PDTN': '11',
 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '1 8 4 0 107 0 0 1 60 1 0 0 255 0 0 3 1 30 2021 3 4 18 0 0 1 0 1 2 1 6 255 0',
 'GRIB_PDS_TEMPLATE_NUMBERS': '1 8 4 0 107 0 0 0 1 0 0 0 60 1 0 0 0 0 0 255 0 0 0 0 0 3 1 30 7 229 3 4 18 0 0 1 0 0 0 0 1 2 1 0 0 0 6 255 0 0 0 0',
 'GRIB_REF_TIME': '  1614643200 sec UTC',
 'GRIB_SHORT_NAME': '0-SFC',
 'GRIB_UNIT': '[kg/(m^2)]',
 'GRIB_VALID_TIME': '  1614880800 sec UTC'}

In [102]:
# Read an specific band: Total Precipation
band = grib.GetRasterBand(53)
band.GetMetadata()

{'GRIB_COMMENT': '06 hr Total precipitation [kg/(m^2)]',
 'GRIB_DISCIPLINE': '0(Meteorological)',
 'GRIB_ELEMENT': 'APCP06',
 'GRIB_FORECAST_SECONDS': '1706400 sec',
 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=2(NCEP Ensemble Products) MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-02-23T00:00:00Z PROD_STATUS=0(Operational) TYPE=4(Perturbed_forecast)',
 'GRIB_PDS_PDTN': '11',
 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '1 8 4 0 107 0 0 1 474 1 0 0 255 0 0 3 1 30 2021 3 15 0 0 0 1 0 1 2 1 6 255 0',
 'GRIB_PDS_TEMPLATE_NUMBERS': '1 8 4 0 107 0 0 0 1 0 0 1 218 1 0 0 0 0 0 255 0 0 0 0 0 3 1 30 7 229 3 15 0 0 0 1 0 0 0 0 1 2 1 0 0 0 6 255 0 0 0 0',
 'GRIB_REF_TIME': '  1614038400 sec UTC',
 'GRIB_SHORT_NAME': '0-SFC',
 'GRIB_UNIT': '[kg/(m^2)]',
 'GRIB_VALID_TIME': '  1615766400 sec UTC'}

In [287]:
seconds = int(band.GetMetadata()['GRIB_VALID_TIME'][2:12])

In [289]:
datetimetiff = datetime(1970,1,1,0,0) + timedelta(0, seconds)

In [292]:
datetimetiff.strftime('%Y-%m-%dZ%H:%M')

'2021-02-27Z09:00'

{'GRIB_COMMENT': 'Temperature [C]',
 'GRIB_DISCIPLINE': '0(Meteorological)',
 'GRIB_ELEMENT': 'TMP',
 'GRIB_FORECAST_SECONDS': '1728000 sec',
 'GRIB_IDS': 'CENTER=7(US-NCEP) SUBCENTER=2(NCEP Ensemble Products) MASTER_TABLE=2 LOCAL_TABLE=1 SIGNF_REF_TIME=1(Start_of_Forecast) REF_TIME=2021-02-23T00:00:00Z PROD_STATUS=0(Operational) TYPE=4(Perturbed_forecast)',
 'GRIB_PDS_PDTN': '1',
 'GRIB_PDS_TEMPLATE_ASSEMBLED_VALUES': '0 0 4 0 107 0 0 1 480 100 0 1000 255 0 0 3 1 30',
 'GRIB_PDS_TEMPLATE_NUMBERS': '0 0 4 0 107 0 0 0 1 0 0 1 224 100 0 0 0 3 232 255 0 0 0 0 0 3 1 30',
 'GRIB_REF_TIME': '  1614038400 sec UTC',
 'GRIB_SHORT_NAME': '1000-ISBL',
 'GRIB_UNIT': '[C]',
 'GRIB_VALID_TIME': '  1615766400 sec UTC'}

In [265]:
band.GetMetadata()['GRIB_REF_TIME'][2:12]

'1613692800'

In [50]:
band

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x7fce39c50a20> >

In [17]:
# Get the band name and description
band = grib.GetRasterBand(145)
metadata = band.GetMetadata()
band_name = metadata['GRIB_COMMENT']
band_description = band.GetDescription()
print(band_name)
print(band_description)

Categorical Rain [0=no; 1=yes]
0[-] SFC="Ground or water surface"


In [124]:
# Lat/lon WSG84 Spatial Reference System
targetPrj = osr.SpatialReference()
targetPrj.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

0

In [196]:
# Read grid data
array = band.ReadAsArray()

In [197]:
array.max()

18.30000114440918

In [198]:
array.min()

0.0

In [189]:
gdal.GetDataTypeName(band.DataType).lower()

'float64'

In [80]:
grib.GetGeoTransform()

(-0.125, 0.25, 0.0, 90.125, 0.0, -0.25)

In [121]:
geotransform = grib.GetGeoTransform()
transform = Affine.from_gdal(*geotransform)

In [122]:
transform

Affine(0.25, 0.0, -0.125,
       0.0, -0.25, 90.125)

# CHANGE GRID AND CROP EXTENT

In [98]:
#min_lon = extent[0]; max_lon = extent[2]; min_lat = extent[1]; max_lat = extent[3]
# Subsect the image
#grib2 = gdal.Translate('subsected_grib.grb', grib, projWin = [min_lon + 360, max_lat, max_lon + 360, min_lat])

In [102]:
nuevo_affine=Affine.identity()
nuevo_affine=nuevo_affine.translation(-360, 90.00000000000001)*nuevo_affine.scale(0.05,-0.05)
nuevo_affine

Affine(0.05, 0.0, -360.0,
       0.0, -0.05, 90.00000000000001)

In [104]:
# Read the GRIB file
#grib = gdal.Open('subsected_grib.grb')

In [232]:
# Read an specific band: Total Precipation
band = grib.GetRasterBand(9)
# Read grid data
array = band.ReadAsArray()

# WRITE GEOTIFF

In [123]:
nw_ds = rasterio.open('test.tiff', 'w', driver='GTiff',
                      height=grib.RasterYSize,
                      width=grib.RasterXSize,
                      count=1, dtype=gdal.GetDataTypeName(band.DataType).lower(),
                      crs=grib.GetProjection(),
                      transform=transform)
nw_ds.write(array, 1)
nw_ds.close()

# Y SI PROBAMOS CON LO DE GOES?

In [294]:
grib.GetProjection()

'GEOGCS["Coordinate System imported from GRIB file",DATUM["unnamed",SPHEROID["Sphere",6371229,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'

In [295]:
grib.GetGeoTransform()

(-0.125, 0.25, 0.0, 90.125, 0.0, -0.25)

In [296]:
# ORIGIN DATASET
# Create grid
origin = memDriver.Create('grid', grib.RasterXSize, grib.RasterYSize, 1, gdal.GDT_Float64)

# Setup projection and geo-transformation
origin.SetProjection(grib.GetProjection())
origin.SetGeoTransform(grib.GetGeoTransform())

0

In [298]:
origin.GetRasterBand(1).WriteRaster(0, 0, grib.RasterXSize, grib.RasterYSize, grib.GetRasterBand(9).ReadRaster())

0

In [237]:
origin

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fce3bbfc690> >

In [299]:
sizex = int((extent[2] - extent[0]) * KM_PER_DEGREE)
sizey = int((extent[3] - extent[1]) * KM_PER_DEGREE)

In [300]:
memDriver = gdal.GetDriverByName('MEM')

# Create grid
grid = memDriver.Create('grid', sizex, sizey, 1, gdal.GDT_Float64)

# Setup projection and geo-transformation
grid.SetProjection(targetPrj.ExportToWkt())
grid.SetGeoTransform(getGeoT(extent, grid.RasterYSize, grid.RasterXSize))

# Perform the projection/resampling

gdal.ReprojectImage(
            origin,
            grid,
            grib.GetProjection(),
            targetPrj.ExportToWkt(),
            gdal.GRA_NearestNeighbour,
            options=['NUM_THREADS=ALL_CPUS']
                        )

# Read grid data
array1 = grid.ReadAsArray()

In [225]:
grid.C

In [228]:
type(band)

NoneType

In [240]:
array1.max()

5.099999904632568

In [301]:
geotransform = grid.GetGeoTransform()
transform = Affine.from_gdal(*geotransform)

In [242]:
transform

Affine(0.008985200845665961, 0.0, -75.0,
       0.0, -0.008987417615338526, -25.0)

In [302]:
# WRITE GIFF
nw_ds = rasterio.open('test_proj.tiff', 'w', driver='GTiff',
                      height=grid.RasterYSize,
                      width=grid.RasterXSize,
                      count=1, dtype=gdal.GetDataTypeName(gdal.GDT_Float64).lower(),
                      crs=grid.GetProjection(),
                      transform=transform)
nw_ds.write(array1, 1)
nw_ds.close()

## FOR GFS

In [318]:
# Read the GRIB file
grib = gdal.Open('../data/GFS/GFS_2021021900+003.grib2')

In [331]:
# Get the band name and description
band = grib.GetRasterBand(145)
metadata = band.GetMetadata()
band_name = metadata['GRIB_COMMENT']
band_description = band.GetDescription()
print(band_name)
print(band_description)

03 hr Total precipitation [kg/(m^2)]
0[-] SFC="Ground or water surface"


In [328]:
for band in range(1, 215):
    metadata = grib.GetRasterBand(band).GetMetadata()
    band_name = metadata['GRIB_COMMENT']
    print(f"band: {band}, {band_name}")

band: 1, u-component of wind [m/s]
band: 2, v-component of wind [m/s]
band: 3, Temperature [C]
band: 4, Temperature [C]
band: 5, Relative humidity [%]
band: 6, u-component of wind [m/s]
band: 7, v-component of wind [m/s]
band: 8, Temperature [C]
band: 9, Relative humidity [%]
band: 10, u-component of wind [m/s]
band: 11, v-component of wind [m/s]
band: 12, Temperature [C]
band: 13, Relative humidity [%]
band: 14, u-component of wind [m/s]
band: 15, v-component of wind [m/s]
band: 16, Temperature [C]
band: 17, Relative humidity [%]
band: 18, u-component of wind [m/s]
band: 19, v-component of wind [m/s]
band: 20, Temperature [C]
band: 21, Relative humidity [%]
band: 22, u-component of wind [m/s]
band: 23, v-component of wind [m/s]
band: 24, Temperature [C]
band: 25, Relative humidity [%]
band: 26, u-component of wind [m/s]
band: 27, v-component of wind [m/s]
band: 28, Temperature [C]
band: 29, Temperature [C]
band: 30, Relative humidity [%]
band: 31, u-component of wind [m/s]
band: 32, v

In [323]:
grib.RasterCount

215

In [332]:
# ORIGIN DATASET
# Create grid
origin = memDriver.Create('grid', grib.RasterXSize, grib.RasterYSize, 1, gdal.GDT_Float64)

# Setup projection and geo-transformation
origin.SetProjection(grib.GetProjection())
origin.SetGeoTransform(grib.GetGeoTransform())
origin.GetRasterBand(1).WriteRaster(0, 0, grib.RasterXSize, grib.RasterYSize, grib.GetRasterBand(145).ReadRaster())

0

In [333]:
sizex = int((extent[2] - extent[0]) * KM_PER_DEGREE)
sizey = int((extent[3] - extent[1]) * KM_PER_DEGREE)

In [334]:
memDriver = gdal.GetDriverByName('MEM')

# Create grid
grid = memDriver.Create('grid', sizex, sizey, 1, gdal.GDT_Float64)

# Setup projection and geo-transformation
grid.SetProjection(targetPrj.ExportToWkt())
grid.SetGeoTransform(getGeoT(extent, grid.RasterYSize, grid.RasterXSize))

# Perform the projection/resampling

gdal.ReprojectImage(
            origin,
            grid,
            grib.GetProjection(),
            targetPrj.ExportToWkt(),
            gdal.GRA_NearestNeighbour,
            options=['NUM_THREADS=ALL_CPUS']
                        )

# Read grid data
array1 = grid.ReadAsArray()

In [335]:
geotransform = grid.GetGeoTransform()
transform = Affine.from_gdal(*geotransform)

In [336]:
# WRITE GIFF
nw_ds = rasterio.open('test_proj_gfs.tiff', 'w', driver='GTiff',
                      height=grid.RasterYSize,
                      width=grid.RasterXSize,
                      count=1, dtype=gdal.GetDataTypeName(gdal.GDT_Float64).lower(),
                      crs=grid.GetProjection(),
                      transform=transform)
nw_ds.write(array1, 1)
nw_ds.close()